In [20]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from collections import OrderedDict
import timm

In [32]:
CP_PATH = "weights/save_pneumonia_imagnet_resnet50"

In [33]:
ckpt = torch.load(CP_PATH + "/model_timm_final.pth")

### Reformat dict keys

In [34]:
new_ckpt = OrderedDict()
for key in ckpt.keys():
    new_key = key[6:]
    new_ckpt[new_key] = ckpt[key]

In [38]:
torch.save(new_ckpt, CP_PATH + "/test_ckpt.pth")

### Load model

In [39]:
model = timm.create_model(model_name="resnet50", checkpoint_path = CP_PATH + "/test_ckpt.pth")